# Generate references for the dataset

These functions aggregate references based on glob strings configurable at various level of the url path (e.g. daily or monthly). This can be farmed out to dask workers and then finally combined using the kerchunk MultiZarrToZarr function.

An example for Australian datasets is shown [here](https://github.com/IOMRC/intake-aodn/blob/main/notebooks/kerchunk_extraction/create_and_aggregate.ipynb)

In [1]:
from dask import delayed, compute
from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=4,threads_per_worker=6)
client = Client(cluster)
client

/opt/conda/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36095 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/pbranson/proxy/36095/status,
Dashboard: /user/pbranson/proxy/36095/status,Workers: 4
Total threads: 24,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44615,Workers: 4
Dashboard: /user/pbranson/proxy/36095/status,Total threads: 24
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:35351,Total threads: 6
Dashboard: /user/pbranson/proxy/35887/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:36521,


# Himawari-SST

You need to break the url into sections, similarly to how we did in Step1.

Example URLs on S3:

```
['noaa-himawari8/AHI-L2-FLDK-SST/2022/01/13/0000/20220113000000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2022/01/13/0100/20220113010000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2022/01/13/0200/20220113020000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2022/01/13/0300/20220113030000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2022/01/13/0400/20220113040000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2022/01/13/0500/20220113050000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2022/01/13/0600/20220113060000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc']
 ```

In [2]:
# Helper functions to process a glob string into an aggregate of references

def process_single(url):
    import fsspec
    from kerchunk.hdf import SingleHdf5ToZarr
    
    s3_fn = 's3://' + url
    with fsspec.open(s3_fn, 
                     anon=True, 
                     mode='rb', 
                     default_fill_cache=False, 
                     default_cache_type='none') as f:
        zarr_dict = SingleHdf5ToZarr(f, s3_fn, spec=1, inline_threshold=100).translate()
        
    return zarr_dict  

def process_aggregate(root='noaa-himawari8/AHI-L2-FLDK-SST/',
                       mask_dict=dict(year='2022',month='01',day='01'),
                       mask_product=
                       mask='{year}/{month}/{day}/{hour}/{year}{month}{day}{hour}00-{tag}-{product}',
                       suffix='_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0',
                       extension='nc',
                       storage_options=dict(anon=True),
                       dest='/home/jovyan/kerchunk_indexes/',
                       dask=True,
                       preprocess=None):
    import fsspec
    import json
    import os
    from kerchunk.combine import MultiZarrToZarr
    
    fs = fsspec.filesystem('s3',use_listings_cache=True,**storage_options)
    
    mask = mask.format(**mask_dict)
    agg_file = f"{root}{''.join(list(mask_dict.values()))}{suffix}.json"
    out_file = os.path.join(dest,agg_file)
    
    print(f'{dest}{root}')
    if not os.path.exists(f'{dest}{root}'):
        print(f'Creating {dest}{root}')
        os.makedirs(f'{dest}{root}')
    
    if not os.path.exists(out_file):
        globstr = f"s3://{root}{mask}*{suffix}.{extension}"
        urls = fs.glob(globstr)

        print(f'Aggregating {globstr} - {len(urls)} found.')

        if len(urls) >= 1:
            so = dict(protocol='s3',
                        profile='default', 
                        default_fill_cache=False, 
                        default_cache_type='first')

            print('Loading references...')
            if dask:
                print('... using dask ...')
                from dask import delayed, compute
                d_process_single = delayed(process_single)
                futures = [d_process_single(u) for u in urls]
                ref_dicts = compute(futures)[0]
            else:
                from tqdm import tqdm 
                ref_dicts = []
                for u in tqdm(urls):
                    ref_dicts.append(process_single(u))

                #setup output location
                
            print(f'Aggregating into {out_file}')

            output = None
            if len(ref_dicts) == 1: # Only one refence in this set... just use the source reference file
                output = json.dumps(ref_dicts[0]).encode()
            else: # otherwise join the references into one file
                mzz = MultiZarrToZarr(ref_dicts,
                                    remote_protocol="s3",
                                    remote_options=storage_options,
                                    concat_dims=["time"], coo_map={"time": "data:time"},
                                    preprocess=preprocess)

                try:
                    dict_agg = mzz.translate()
                    output = json.dumps(dict_agg).encode()
                except Exception as ex:
                    agg_file = f'ERROR(UNKOWN): {agg_file} {str(ex)}'
                    raise ex

            if not output is None:
                with open(out_file,"wb") as outf:
                    print('writing')
                    outf.write(output)

            return {mask: agg_file}
        else:
            return {mask: 'ERROR(NOFILES)'}
    return {mask:'existing'}

In [3]:
process_aggregate()

/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/


{'2022/01/01/*/20220101': 'existing'}

In [4]:
def open_single(fn,preprocess=None,storage_options=dict(anon=True)):
    import fsspec
    import xarray as xr
    
    mapper=fsspec.get_mapper('reference://',
                             fo=fn,
                             remote_protocol='s3',
                             remote_options=storage_options,
                            )
    ds = xr.open_zarr(mapper,chunks={}, consolidated=False, decode_times=False)   
    
    if preprocess is not None:
        ds = preprocess(ds)
    
    return ds

ds = open_single('/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/20220101-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json')
ds

,Array,Chunk
Bytes,14.48 GiB,24.72 MiB
Shape,"(24, 9000, 18000)","(1, 1800, 3600)"
Count,601 Tasks,600 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,14.48 GiB,24.72 MiB
Shape,"(24, 9000, 18000)","(1, 1800, 3600)"
Count,601 Tasks,600 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [5]:
ds.nbytes / 1E9

264.384108384

# Map out across full dataset

In [6]:
import fsspec
fs = fsspec.filesystem('s3',anon=True)
fs.ls('noaa-himawari8/AHI-L2-FLDK-SST/')

['noaa-himawari8/AHI-L2-FLDK-SST/2019',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020',
 'noaa-himawari8/AHI-L2-FLDK-SST/2021',
 'noaa-himawari8/AHI-L2-FLDK-SST/2022']

In [7]:
fs.ls('noaa-himawari8/AHI-L2-FLDK-SST/2020/01')

['noaa-himawari8/AHI-L2-FLDK-SST/2020/01/01',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/02',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/03',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/04',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/05',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/06',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/07',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/08',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/10',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/11',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/12',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/13',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/14',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/15',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/16',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/17',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/18',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/19',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/20',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/21',
 'noaa-himawari8/AHI-L2-FLDK-SST/2020/01/22',
 'noaa-himawari8/AHI-L2-FLDK-SST/2

In [9]:
fs.glob('noaa-himawari8/AHI-L2-FLDK-SST/2021/04/07/1100/*')

['noaa-himawari8/AHI-L2-FLDK-SST/2021/04/07/1100/20210407110000-NCCF-L2P_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2021/04/07/1100/20210407110000-NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc',
 'noaa-himawari8/AHI-L2-FLDK-SST/2021/04/07/1100/ACSPO_V2.80_H08_AHI_2021-04-07_1100-1110_20210407.123412.nc']

In [11]:
kwargs = dict(root='noaa-himawari8/AHI-L2-FLDK-SST/',
               mask_dict=dict(year='2022',month='01',tag='STAR'),
               mask='{year}/{month}/*/*/{year}{month}*{tag}',
               suffix='-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0',
               extension='nc',
               storage_options=dict(anon=True),
               dest='/home/jovyan/kerchunk_indexes/',
               dask=True,
               preprocess=None)

In [12]:
%%time
# Make monthly aggregates
import pandas as pd

dt = pd.date_range('2020-01-01','2021-04-07',freq='M')

results = []
for d in dt:
    print(d)
    kwargs['mask_dict'] = dict(year=d.strftime('%Y'), 
                               month=d.strftime('%m'),
                               tag='STAR'
                              )
    results.append(process_aggregate(**kwargs))


2020-01-31 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-himawari8/AHI-L2-FLDK-SST/2020/01/*/*/202001*STAR*-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc - 0 found.
2020-02-29 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-himawari8/AHI-L2-FLDK-SST/2020/02/*/*/202002*STAR*-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc - 0 found.
2020-03-31 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-himawari8/AHI-L2-FLDK-SST/2020/03/*/*/202003*STAR*-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc - 0 found.
2020-04-30 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-himawari8/AHI-L2-FLDK-SST/2020/04/*/*/202004*STAR*-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc - 0 found.
2020-05-31 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-

In [ ]:
%%time
# Make monthly aggregates
import pandas as pd

dt = pd.date_range('2021-04-01','2022-09-01',freq='M')

results = []
for d in dt:
    print(d)
    kwargs['mask_dict'] = dict(year=d.strftime('%Y'), 
                               month=d.strftime('%m'),
                               tag='NCCF'
                              )
    results.append(process_aggregate(**kwargs))

2021-04-30 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-himawari8/AHI-L2-FLDK-SST/2021/04/*/*/202104*NCCF*-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc - 584 found.
Loading references...
... using dask ...
Aggregating into /home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202104NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json
writing
2021-05-31 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-himawari8/AHI-L2-FLDK-SST/2021/05/*/*/202105*NCCF*-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.nc - 736 found.
Loading references...
... using dask ...
Aggregating into /home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/202105NCCF-L3C_GHRSST-SSTsubskin-AHI_H08-ACSPO_V2.80-v02.0-fv01.0.json
writing
2021-06-30 00:00:00
/home/jovyan/kerchunk_indexes/noaa-himawari8/AHI-L2-FLDK-SST/
Aggregating s3://noaa-himawari8/AHI-L2-FLDK-SST/2021/06/*/*/202106*NCC